### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [1]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

2023-05-17 13:52:10.063348: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 13:52:11.092430: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Hyperparameter space definition

In [2]:
hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    # 'x2': ["interval", -2, 8],
    # 'x3': ["interval", -2, 8],
    # 'x4': ["interval", -2, 8],
    # 'x5': ["interval", -2, 8],
    # 'x6': ["interval", -2, 8],
    # 'x7': ["interval", -2, 8],
    # 'x8': ["interval", -2, 8],
    # 'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [3]:
DIRECTORY = "Current_tests/"+time.strftime("%H_%M_%S", time.localtime())

SPARSE_PARAMS = [5, 0.85, "gradient_descent"]

BUDGETS = [50, 100, 200, 300, 400, 500, 750, 1000, 1500, 2000, 3000, 4000, 5000]

RESULTS_normal = "{"


### Optimization

In [4]:
# os.mkdir(DIRECTORY)

# f = open(DIRECTORY+"/configurations.txt", "a")
# f.write("Dimension of hyperparameter space: " +
#         str(len(hyperparameterspace)) + "\n")
# for key in hyperparameterspace.keys():
#     f.write("\n" + key + ": " + str(hyperparameterspace[key]))
# f.write("\n\nSparse grid degree: " + str(SPARSE_PARAMS[0]))
# f.write("\nSparse grid adaptivity param: " + str(SPARSE_PARAMS[1]))
# f.write("\nSparse grid optimization alg: " + str(SPARSE_PARAMS[2]))
# f.write("\nBUDGETS: " + str(BUDGETS))
# f.close()


################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(x):
    sum = 0
    for i in range(len(x)):
        sum += x[i]**2 - 10 * math.cos(2 * math.pi * x[i])
    return len(hyperparameterspace) * 10 + sum


class ExampleFunction(pysgpp.ScalarFunction):

    def __init__(self):
        super(ExampleFunction, self).__init__(
            len(hyperparameterspace.keys()))

    def eval(self, x):

        x_temp = []
        for i in range(len(x)):
            current = "x" + str(i)
            lower = hyperparameterspace_special[current][0]
            upper = hyperparameterspace_special[current][1]
            x_temp.append(HPO.from_standard(lower, upper, x[i]))

        return evaluate_model(x_temp)

hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    # 'x2': ["interval", -2, 8],
    # 'x3': ["interval", -2, 8],
    # 'x4': ["interval", -2, 8],
    # 'x5': ["interval", -2, 8],
    # 'x6': ["interval", -2, 8],
    # 'x7': ["interval", -2, 8],
    # 'x8': ["interval", -2, 8],
    # 'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste

#### For each dataset: run models with different budget #####
for BUDGET in BUDGETS:

    print("\n################################################## Current Budget:",
            BUDGET, "##################################################")

    ########################### SPARSE OPT ############################

    print("\nPerforming sparse search")

    f = ExampleFunction()
    dataset = HPO.Dataset([], [])

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=BUDGET, verbosity=VERBOSE, degree=5, adaptivity=0.85)

    [x0, fX0, x1, fX1, x2, fX2], cost  = optimization.fit()

    fXOpt = fX0

    if fX1 < fXOpt:
        fXOpt = fX1 

    if fX2 < fXOpt:
        fXOpt = fX2


    RESULTS_normal += "(" + str(cost) + "," + str(fXOpt) + ") "



RESULTS_normal += "}"


print(RESULTS_normal)


hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    'x2': ["interval", -2, 8],
    'x3': ["interval", -2, 8],
    # 'x4': ["interval", -2, 8],
    # 'x5': ["interval", -2, 8],
    # 'x6': ["interval", -2, 8],
    # 'x7': ["interval", -2, 8],
    # 'x8': ["interval", -2, 8],
    # 'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste

#### For each dataset: run models with different budget #####
for BUDGET in BUDGETS:

    print("\n################################################## Current Budget:",
            BUDGET, "##################################################")

    ########################### SPARSE OPT ############################

    print("\nPerforming sparse search")

    f = ExampleFunction()
    dataset = HPO.Dataset([], [])

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=BUDGET, verbosity=VERBOSE, degree=5, adaptivity=0.85)

    [x0, fX0, x1, fX1, x2, fX2], cost  = optimization.fit()

    fXOpt = fX0

    if fX1 < fXOpt:
        fXOpt = fX1 

    if fX2 < fXOpt:
        fXOpt = fX2


    RESULTS_normal += "(" + str(cost) + "," + str(fXOpt) + ") "



RESULTS_normal += "}"


print(RESULTS_normal)



hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    'x2': ["interval", -2, 8],
    'x3': ["interval", -2, 8],
    'x4': ["interval", -2, 8],
    'x5': ["interval", -2, 8],
    # 'x6': ["interval", -2, 8],
    # 'x7': ["interval", -2, 8],
    # 'x8': ["interval", -2, 8],
    # 'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste

#### For each dataset: run models with different budget #####
for BUDGET in BUDGETS:

    print("\n################################################## Current Budget:",
            BUDGET, "##################################################")

    ########################### SPARSE OPT ############################

    print("\nPerforming sparse search")

    f = ExampleFunction()
    dataset = HPO.Dataset([], [])

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=BUDGET, verbosity=VERBOSE, degree=5, adaptivity=0.85)

    [x0, fX0, x1, fX1, x2, fX2], cost  = optimization.fit()

    fXOpt = fX0

    if fX1 < fXOpt:
        fXOpt = fX1 

    if fX2 < fXOpt:
        fXOpt = fX2


    RESULTS_normal += "(" + str(cost) + "," + str(fXOpt) + ") "



RESULTS_normal += "}"


print(RESULTS_normal)





hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    'x2': ["interval", -2, 8],
    'x3': ["interval", -2, 8],
    'x4': ["interval", -2, 8],
    'x5': ["interval", -2, 8],
    'x6': ["interval", -2, 8],
    'x7': ["interval", -2, 8],
    # 'x8': ["interval", -2, 8],
    # 'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste

#### For each dataset: run models with different budget #####
for BUDGET in BUDGETS:

    print("\n################################################## Current Budget:",
            BUDGET, "##################################################")

    ########################### SPARSE OPT ############################

    print("\nPerforming sparse search")

    f = ExampleFunction()
    dataset = HPO.Dataset([], [])

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=BUDGET, verbosity=VERBOSE, degree=5, adaptivity=0.85)

    [x0, fX0, x1, fX1, x2, fX2], cost  = optimization.fit()

    fXOpt = fX0

    if fX1 < fXOpt:
        fXOpt = fX1 

    if fX2 < fXOpt:
        fXOpt = fX2


    RESULTS_normal += "(" + str(cost) + "," + str(fXOpt) + ") "



RESULTS_normal += "}"


print(RESULTS_normal)





hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    'x2': ["interval", -2, 8],
    'x3': ["interval", -2, 8],
    'x4': ["interval", -2, 8],
    'x5': ["interval", -2, 8],
    'x6': ["interval", -2, 8],
    'x7': ["interval", -2, 8],
    'x8': ["interval", -2, 8],
    'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste

#### For each dataset: run models with different budget #####
for BUDGET in BUDGETS:

    print("\n################################################## Current Budget:",
            BUDGET, "##################################################")

    ########################### SPARSE OPT ############################

    print("\nPerforming sparse search")

    f = ExampleFunction()
    dataset = HPO.Dataset([], [])

    optimization = HPO.SparseGridSearchOptimization(
        dataset, f, hyperparameterspace, budget=BUDGET, verbosity=VERBOSE, degree=5, adaptivity=0.85)

    [x0, fX0, x1, fX1, x2, fX2], cost  = optimization.fit()

    fXOpt = fX0

    if fX1 < fXOpt:
        fXOpt = fX1 

    if fX2 < fXOpt:
        fXOpt = fX2


    RESULTS_normal += "(" + str(cost) + "," + str(fXOpt) + ") "



RESULTS_normal += "}"


print(RESULTS_normal)



################################################## Current Budget: 50 ##################################################

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 1ms.
Solving linear system (automatic method)...
Done in 0ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 1ms.

################################################## Current Budget: 100 ##################################################

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 2ms.
Solving linear system (automatic method)...
Done in 1ms.
Optimizing (gradient descent)...
Done in 0ms.
Optimizing (multi-start)...
Done in 2ms.

################################################## Current Budget: 200 ##################################################

Performing sparse search
Adaptive grid generation (Ritter-Novak)...
Done in 4ms.
Solving linear system (automatic method)...
Done in 10ms.
Optimizing (gradient descent)...
Done in